<a href="https://colab.research.google.com/github/gouravsinha1405/finetuning-Falcon-qlora/blob/main/falcon_finetune_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [2]:
!nvidia-smi

Fri Jun 23 10:27:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Installations

In [ ]:
!pip install -Uqqq pip --progress-bar off
!pip install spacy-experimental --progress-bar off
!pip install https://github.com/explosion/spacy-experimental/releases/download/v0.6.0/en_coreference_web_trf-3.4.0a0-py3-none-any.whl --progress-bar off
!spacy download en_core_web_trf --progress-bar off
!pip install bitsandbytes --progress-bar off
!pip install torch --progress-bar off
!pip install git+https://github.com/huggingface/transformers@de9255de27abfcae4a1f816b904915f0b1e23cd9 --progress-bar off
!pip install -U git+https://github.com/huggingface/peft.git --progress-bar off
!pip install accelerate==0.20.3 --progress-bar off
!pip install loralib --progress-bar off
!pip install einops --progress-bar off


## Imports

In [ ]:
import json
import os
import bitsandbytes as bnb
import pandas as pd
import torch
import transformers
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

In [5]:
notebook_login()

In [4]:
PEFT_MODEL = r"gouravsinha/falcon-financial-NER"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from main import FinanceNER
pipe = FinanceNER(model)

## Examples

In [7]:
text = """Mumbai: India's largest drone manufacturing company, IdeaForge Technology, has completed its pre-IPO (initial public offering) placement at a valuation of around ₹2,500 crore, banking sources told ET. The round, which raised ₹60 crore, saw participation from institutional investors, including Tata AIG General Insurance, 360 One Asset Management, Motilal Oswal Mutual Fund, and Think Investments PCC, a foreign institutional investor, said bankers . IdeaForge, according to bankers, filed its red herring prospectus earlier this week. The company is expected to launch its public issue at the end of this month. The issue size is estimated to be around ₹650 crore with an offer for sale (OFS) component of around ₹350 crore and fresh fundraising of ₹300 crore. Under the OFS, promoter Ashish Bhat will sell 180,000 shares, Amarpreet Singh will offload 8,362 shares, and Nambirajan Seshadri will sell 22,600 shares. Other selling shareholders include A&E Investment LLC, Agarwal Trademart, and Celesta Capital II Mauritius. IdeaForge filed its draft red herring prospectus (DRHP) with Sebi in February this year and recently received its approval. The company has been backed by several marquee ventures and private equity investors, including Qualcomm Asia, Infosys , and Celesta Capital. Mathew Cyriac-backed Florintree Enterprises is the single largest shareholder in the company with approximately 12% stake. Proceeds from its fresh issuance will be utilised to repay certain indebtedness availed by the company, fund working capital requirements, and invest in product development and general corporate purposes. IdeaForge is a market leader in India's unmanned aircraft systems (UAS) market, with a market share of approximately 50% in fiscal 2022. The company was incubated at IIT Bombay in 2007 and is one of the first few players in manufacturing commercial drones in India. The company is one of the leading players in manufacturing drones in the dual-use category (civil and defen"""

In [8]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......
*******************************************
input sentence :  Under the OFS , promoter Ashish Bhat will sell 180,000 shares , Amarpreet Singh will offload 8,362 shares , and Nambirajan Seshadri will sell 22,600 shares .
output:  [{'Name': 'Amarpreet Singh', 'Designation': 'NA', 'Companies': 'NA', 'Shares Bought/Sold': '8,362', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'Nambirajan Seshadri', 'Designation': 'NA', 'Companies': 'NA', 'Shares Bought/Sold': '22,600', 'Type of Shares': 'NA', 'Acquisitions':,[{'Name': 'Ashish Bhat', 'Designation': 'Promoter', 'Companies': 'NA', 'Shares Bought/Sold': '180,000', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]
********************************************
*******************************************
input sentence :  Mathew Cyriac - backed Florintree Enterprises is the single largest shareholder in the 

"[{'Name': 'Amarpreet Singh', 'Designation': 'NA', 'Companies': 'NA', 'Shares Bought/Sold': '8,362', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'Nambirajan Seshadri', 'Designation': 'NA', 'Companies': 'NA', 'Shares Bought/Sold': '22,600', 'Type of Shares': 'NA', 'Acquisitions':,[{'Name': 'Ashish Bhat', 'Designation': 'Promoter', 'Companies': 'NA', 'Shares Bought/Sold': '180,000', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'Mathew Cyriac', 'Designation': 'NA', 'Companies': 'Florintree Enterprises', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'Mathew Cyriac', 'Designation': 'NA', 'Companies': 'Florintree Enterprises', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions':"

In [9]:
text = """Mumbai: Four state-owned financial entities - Punjab National Bank , Life Insurance Corporation of India , State Bank of India , and Bank of Baroda - that together own just over 45% of UTI Asset Management Company (AMC) are understood to be working on inviting formal bids for selling their stakes, top officials close to the development told ET. Industry sources said these entities have recently appointed merchant bankers to initiate the sale process. Earlier, the Tata Group was understood to be in advanced talks to buy a majority stake in UTI AMC from the four public sector entities, but the discussions fell through after a large investor in the AMC insisted that the other investors follow a formal bidding process to sell their stakes instead of using the nomination route, said one of the officials cited above. There were also some issues that had to be sorted around the stakes held, it is learnt. The Tata Group will now put in a fresh bid for the AMC, officials close to the development said. In September last year, the Department of Investment and Public Asset Management (Dipam) issued detailed guidelines for public sector companies on dealing with their downstream investments. While giving state entities greater autonomy, the government has asked the respective boards to follow thorough due diligence on qualified bidders, including assessment of security risks and political clearance of qualified bidders if the latter is necessary from a national security standpoint. Not Classified as Promoters ET's mailed queries to UTI AMC, Punjab National Bank, Life Insurance Corporation of India, State Bank of India, Bank of Baroda and the Tata Group remained unanswered. Although UTI AMC had earlier denied talks of a stake sale by the four public-sector sponsors, the Tata Group is keen on buying the 45% stake from the four state-owned financial entities as its current asset management business is very small. To be a significant player in the segment, the Tata Group must scale up, and the merger & acquisition route seems to be the only way to do so quickly, one of the officials cited above said. Purse & Purpose "It is mostly the Tata Group, which has a serious intent to scale up its asset management business and (has) the deep pockets to fund such an acquisition," said a highly placed official at a leading mutual fund. UTI AMC, which was carved out of the erstwhile Unit Trust of India (UTI) in 2003, has no promoters but sponsors, such as the four entities interested in selling their stakes. The other large investor in the AMC is T Rowe Price Group Inc which, through its wholly owned subsidiary T Rowe Price International, held a 22.96% stake in UTI AMC as on March 31, 2023. The public-sector entities, which together own 45%, are not classified as promoters of the AMC because they also have their own asset management businesses, and Securities and Exchange Board of India (Sebi) rules bar companies from being promoters of more than one asset manager. Furthermore, India's takeover rules mandate the acquirer of 25% stake in a publicly listed company to mandatorily make an open offer to other shareholders for buying an additional 26% equity. Hence, a single buyer of the entire 45% stake will be ordinarily required to make an open offer for an additional 26%. UTI AMC, which was listed in October 2020, commands a market capitalisation of ₹8,800 crore as of Wednesday. Assets under management (AUM) of UTI AMC, as of March 31, 2023, were ₹2.39 lakh crore, or about 6% of the total industry AUM. Shares of UTI AMC declined 11% in the past six months, compared with gains of half a percent in the benchmark Nifty. For the year ended March 31, 2023, UTI AMC posted an 18% year-on-year decline in its net profit at Rs 440 crore on revenue of Rs 1,267 crore. Last year, Punjab National Bank said it wanted to monetise its non-core assets, including selling its stake in UTI AMC. In October 2020, the lender sold a 3% stake in UTI AMC for around ₹180 crore, bringing down ownership to 15.22%. In December 2019, Sebi directed LIC , SBI , and BoB to reduce their stakes in UTI AMC by December 2020. These three public sector financial institutions held 18.24% each back then, while Sebi's cross-holding norms for mutual funds do not allow the sponsor of one AMC to hold more than 10% in another. Subsequently, these three firms reduced their stakes below 10% during the IPO in October 2020. Don’t"""

In [10]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......


''

In [11]:
text = """A significant percentage of companies in India believe the advent of 5G will herald a transformative era in the world of work, unlocking a wealth of opportunities for India's tech talent pool, says a report. According to the report by staffing company TeamLease Services , over 80 per cent of respondents believe that IT and banking and Financial Services will benefit from the 5G technology in terms of job creation and upskilling. The report titled "India's 5G rollout: Revolutionising the People Supply Chain" is based on responses of over 247 companies on the impact of 5G on the ecosystem and employment generation. "With substantial investments, such as the PLI scheme allocating Rs 12,000 crore to the telecom sector, with approximately 25 per cent dedicated to creating new jobs, we are poised for a positive outlook on job creation and upskilling," TeamLease Services CEO - Staffing, Kartik Narayan said. This will boost confidence to harness the potential of 5G, creating unprecedented job opportunities, driving innovation and shaping a transformative future, he added. The adoption of 5G will have significant impact on industries such as BFSI (60 per cent), education (48 per cent), gaming (48 per cent) and retail and e-commerce (46 per cent), the report said. On a year-on-year basis, 46 per cent of participants anticipate job creation ranging from 61 per cent to 80 per cent within the first year of 5G implementation. Discover the stories of your interest Blockchain 5 Stories Cyber-safety 7 Stories Fintech 9 Stories E-comm 9 Stories ML 8 Stories Edtech 6 Stories "Telecommunications companies embracing 5G solutions present a promising growth opportunity. As telcos leverage the potential of 5G in network upgradation, migration, IoT/IIoT, mobility, network infrastructure, and network security, we anticipate increased hiring for specialised roles," Narayan said. This advancement creates new opportunities and drives demand for roles such as technical content writers, networking engineers, AI/ML professionals, UX designers, cloud computing engineers, cybersecurity specialists, data science and analytical experts and more, he added. The report, however, noted that the telecom industry is facing a considerable demand-supply gap of 28 per cent, which emphasises the urgent need for comprehensive upskilling and reskilling initiatives. Encouragingly, 88 per cent of employees expressed a preference for upskilling, indicating a strong desire to enhance their existing skill sets, the report said. Additionally, 11.2 per cent of employees showed a preference for reskilling, presenting significant opportunities in the job market for fresh talent, it added. The report also highlighted the impact of 5G on job creation over the next few years, with 41 per cent of respondents expecting an impact of more than 80 per cent on job creation over the next three ye"""

In [12]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......
*******************************************
input sentence :  " With substantial investments , such as the PLI scheme allocating Rs 12,000 crore to the telecom sector , with approximately 25 per cent dedicated to creating new jobs , we are poised for a positive outlook on job creation and upskilling , " TeamLease Services CEO - Staffing , Kartik Narayan said .
output:  [{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]
### Response Type:
[{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]
### Response Type:
[{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sol

"[{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]\n### Response Type:\n[{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]\n### Response Type:\n[{'Name': 'Kartik Narayan', 'Designation': 'CEO - Staffing', 'Companies': 'TeamLease Services', 'Shares Bought/Sold': 'NA', '"

In [13]:
text = """Analysts believe IKIO Lighting will list at a strong premium on June 16 given the premium at which it trades in the grey market, demand for the shares during book building and strong outlook of its business in the long term.

“Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an upside listing at around 30–35 percent, and the company's current performance and growth potential justify this gain,” said Anubhuti Mishra, Equity Research Analyst at Swastika Investmart Ltd.

“Secondly, its strong focus on R&amp;D and backward integration, its well-established relationship with clients, and its consistent financial performance are also some reasons why this IPO is attracting such demand.”

Incidentally, the grey market premium came down in the last few days, which hit a peak of Rs 125 or 44 percent over the issue price. It fell amid consolidation in the secondary market. However, analysts are unperturbed.

“We are expecting the company to list at 25-30 percent premium,” said Astha Jain, Senior Research Analyst at Hem Securities. “We recommend partial profit booking on the listing day if it trades at 30-35 percent premium.”

She advised investors, who got the allocation to hold partial allotment for the long-term as the company is well poised to capture growth of the LED market with a diverse product basket and focus on high-margin areas.

Among the key strengths of the company, analysts highlight, are long-term relationships with leading industry customers, a strong focus on R&amp;D, established infrastructure with backward integration along with strong and consistent financial performance.

The maiden public issue of Noida-based IKIO was subscribed 66.30 times during June 6-8 with qualified institutional buyers' (QIBs) reserved portion being booked 163.58 times, high networth individuals (HNIs) 63.35 times and retail 13.86 times.

“IKIO Lighting has attracted strong interest from market participants and is expected to list at a significant premium to the issue price,” said Manish Chowdhury Head of Research Stoxbox. “We attribute this robust expectation to the company’s adoption of the ODM business model which aids it to maintain long-term relationships with customers and is margin accretive due to higher premiums charged for its designs.”

He added that the company’s focus to diversify into newer products along with the high growth potential of the Indian high-end home and decorative lighting market bodes well for the company’s future business prospects.

Its products are categorised as LED lighting, refrigeration lights, ABS (acrylonitrile butadiene styrene) piping and other products.

IKIO and its shareholders raised Rs 606.5 crore via public issue, at the upper end of the price band of Rs 270-285 per share, comprising a fresh issue of Rs 350 crore.

(With inputs from Sunil Shankar Matkar)

Disclaimer: The views and investment tips expressed by investment experts on Moneycontrol.com are their own and not those of the website or it"""

In [14]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......
*******************************************
input sentence :  “ Considering the overwhelming response the IKIO IPO received and its current grey market premium , we can expect an upside listing at around 30–35 percent , and the company 's current performance and growth potential justify this gain , ” said Anubhuti Mishra , Equity Research Analyst at Swastika Investmart Ltd.
output:  [“ Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an upside listing at around 30–35 percent, and the company's current performance and growth potential justify this gain, ” said Anubhuti Mishra, Equity Research Analyst at Swastika Investmart Ltd. ]
### Response Type:
List
### Response Value:
[“ Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an upside

"[“ Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an upside listing at around 30–35 percent, and the company's current performance and growth potential justify this gain, ” said Anubhuti Mishra, Equity Research Analyst at Swastika Investmart Ltd. ]\n### Response Type:\nList\n### Response Value:\n[“ Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an upside listing at around 30–35 percent, and the company's current performance and growth potential justify this gain, ” said Anubhuti Mishra, Equity Research Analyst at Swastika Investmart Ltd. ]\n### Response Type:\nList\n### Response Value:\n[“ Considering the overwhelming response the IKIO IPO received and its current grey market premium, we can expect an,[{'Name': 'Astha Jain', 'Designation': 'Senior Research Analyst', 'Companies': 'Hem Securities', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitio

In [16]:
text = """n a bid to cut carbon emission and achieve net zero carbon emission by 2045 , Tata Steel signed an agreement with SMS Group on Wednesday. Indian steel major has signed this deal with the German company to collaborate on a technology that may cut carbon emission by more than 50 per cent during steel making process, the steel maker claimed in a statement. As part of the agreement, the two companies will hold technical discussions and demonstrate a decarbonisation technology developed by the SMS group at a blast furnace in Tata's Jamshedpur plant, Tata Steel said. Tata Steel went on to add that major goal of this deal is to reduce carbon dioxide emissions by more than 50% from the blast furnace's baseline operation. Speaking on the Tata Steel and SMS Group deal, TV Narendran, CEO at Tata Steel said, “India being the second largest steel producer in the world also places a huge responsibility on large manufacturers like Tata Steel to lead the country's decarbonisation journey." Steel production generates up to 9% of global carbon emissions, according to the World Steel Association, and producers across the globe are investing in technology that can help reduce their share of pollution. Among other major sources of carbon emission, the IEA (International Energy Agency) estimates that direct CO2 emissions due to crude steel production is approximately 1.4 tons CO2 per ton steel produced. Recycling today claims the numbers are slightly higher, roughly 1.85 metric tons of CO2 per ton steel. The IEA base these findings on the basis of a report from World Steel Association. McKinsey has also raised concern over the rising carbon emission by various sources. It also base the World Steel Association report over the carbon emission data. Being the largest producer of steel in the world, China is largest contributor in carbon emission as well. According to experts, major reason for high emission by China is ‘Blast Furnace Ovens’, which is widely used in Chinese steel plants. (With inputs from Reuters) ABOUT THE AUTHOR Asit Manohar Chief Content Producer at Live Mint"""

In [17]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......
*******************************************
input sentence :  Speaking on the Tata Steel signed   SMS Group deal , TV Narendran , CEO at Tata Steel signed   , “ India being the second largest steel producer in the world also places a huge responsibility on large manufacturers like Tata Steel signed   lead the country 's decarbonisation journey . "
output:  [{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]
### Response Type:
[{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]
### Response Type:
[{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}
********************

"[{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]\n### Response Type:\n[{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]\n### Response Type:\n[{'Name': 'TV Narendran', 'Designation': 'CEO', 'Companies': 'Tata Steel', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'},[{'Name': 'Asit Manohar', 'Designation': 'Chief Content Producer', 'Companies': 'Live Mint', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'Asit Manohar', 'Designation': 'Chief Content Producer', 'Companies': 'Live Mint', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions':"

In [18]:
text = """A first-of-its-kind topical gel for the treatment of erectile dysfunction will be hitting store shelves soon, no prescription needed! Pharmaceutical company Futura Medical, which developed the product, informed, topical gel named Eroxon has received authorization for over-the-counter marketing in the United States. The company claims, it is the first topical erectile dysfunction treatment available without the need for a prescription. Meanwhile, US Food and Drug Administration confirmed that Eroxon has been granted a "de novo" classification by the US Food and Drug Administration. This classification recognizes Eroxon as a new type of product. “FDA set a very high standard in evaluating the effectiveness and safety of De Novo Medical Devices. I am delighted that we met this standard," James Barder, chief executive officer of Futura Medical, said in a news release Monday. Eroxon is already available in Belgium and the United Kingdom. In the UK, a four-pack of Eroxon costs about £24.99, a Futura Medical spokesperson said Monday. That’s roughly about $31.22 or ₹ 2,500. However, the prices in the US are still not decided and spokesperson said, as reported by CNN , ‘ it will ultimately be determined by the partner who launches Eroxon in the US’ Also, the availability of Eroxon in US stores depends on a partner but some analysts predict a possible launch in 2025. A leaflet for the product notes that the gel comes in a single-dose tube and should be applied to the head of the penis for 15 seconds before sex. Marketing materials for the product claim that it can help users get an erection within 10 minutes of application. The erection lasts long enough for successful sex in about 65% of people and should naturally subside, according to Eroxon’s website. Erectile dysfunction affects approximately 30 million men in the US, causing difficulty in achieving or maintaining a firm erection for satisfactory sexual activity. Contributing factors include medical conditions (e.g., diabetes), medications (e.g., antidepressants), psychological factors, and unhealthy behaviors (e.g., smoking, excessive alcohol consumption, obesity). (With inputs from CNN) ABOUT THE AUTHOR Sanchari Ghosh Sanchari Ghosh is a Chief Con"""

In [19]:
pipe(text)

performing coref resolution.....
splitting text into sentences .....
discarding irrelevant sentences ......
generating output .......
*******************************************
input sentence :  I am delighted that we met this standard , " James Barder , chief executive officer of Futura Medical , said in a news release Monday .
output:  [{'Name': 'James Barder', 'Designation': 'chief executive officer', 'Companies': 'Futura Medical', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'James Barder', 'Designation': 'chief executive officer', 'Companies': 'Futura Medical', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions':
********************************************
*******************************************
input sentence :  ABOUT THE AUTHOR Sanchari Ghosh Sanchari  is a Chief Con
output:  [{'Name': 'Sanchari Ghosh', 'Designation': 'Chief Con', 'Companies': 'NA', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'N

"[{'Name': 'James Barder', 'Designation': 'chief executive officer', 'Companies': 'Futura Medical', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}],[{'Name': 'James Barder', 'Designation': 'chief executive officer', 'Companies': 'Futura Medical', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions':,[{'Name': 'Sanchari Ghosh', 'Designation': 'Chief Con', 'Companies': 'NA', 'Shares Bought/Sold': 'NA', 'Type of Shares': 'NA', 'Acquisitions': 'NA'}]"